In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ai-defence-summer-school-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F82126%2F8962580%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240628%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240628T092926Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D91f65fdcee2007b2d56694a853c9c79b8797cff46ad62f2ca8900aff20982b867377fafc18d2f370f5800df1acf6dc2f3c709bea8cd4a5cb597ef69aba98f8079c1de80d36c9dce2a1b781fb58180e6cebfcc2a1bacd9b1240a52aa4c54aed522ae4ba9cb634ccdafd3eb14fbc117b3b46449570267a24ccf2286cf22774308ecf222c1e940bc34481adcc77370d5b0554892a3d74cbbfa64b7fa6bef9117bb38d63d7799d043073e17867e0448466b61bda2c228e0b37d523d3399aa44f627413ec266da4e6b24eec2000ca5cb4b78c40ac7221dd5112274070aa5ce43d8dceb069e73259d7877eeec207695094982986686ec7b6422dfe33b49e2772504ed1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 141977907 bytes downloaded
Downloaded and uncompressed: ai-defence-summer-school-2024
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-defence-summer-school-2024/test.csv
/kaggle/input/ai-defence-summer-school-2024/sample_submission.csv
/kaggle/input/ai-defence-summer-school-2024/train.csv
/kaggle/input/ai-defence-summer-school-2024/messages/messages.jsonl


## Read input data (merging metadata with messages + checking NaN)

In [ ]:
# train dataframe contains source information and source category
train_df = pd.read_csv('/kaggle/input/ai-defence-summer-school-2024/train.csv')
# test dataframe contains source information only, and you need to recreate source category
test_df = pd.read_csv('/kaggle/input/ai-defence-summer-school-2024/test.csv')
# messages dataframe contains all the available message for all sources across train and test dataframes
messages_df = pd.read_json('/kaggle/input/ai-defence-summer-school-2024/messages/messages.jsonl', lines=True)

In [ ]:
messages_text_only = messages_df[['source_id', 'text']]

In [ ]:
train_df_text = pd.merge(messages_text_only, train_df, on='source_id')
test_df_text = pd.merge(messages_text_only, test_df, on='source_id')
print(train_df_text.shape, test_df_text.shape)

(263353, 4) (109194, 3)


In [ ]:
# drop duplicates
train_df_text = train_df_text.drop_duplicates(subset=['text'])
# test_df_text = test_df_text.drop_duplicates(subset=['text'])
print(train_df_text.shape, test_df_text.shape)

(242391, 4) (109194, 3)


In [ ]:
print("NaN among text and category in train: ", (train_df_text["text"].isna() & train_df_text["source_category"].isna()).sum())
print("NaN among text in test: ", (test_df_text["text"].isna().sum()))

NaN among text and category in train:  0
NaN among text in test:  0


# LogReg / Gradient Boosting + TfIdf

In [ ]:
import nltGradientBoostingClassifierk
from nltk.corpus import stopwords
import string
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nltk.download('stopwords')
stop_words = list(set(stopwords.words('russian')))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess_text(text):
    # Remove links
    url_pattern = re.compile(r'https?://\S+|www\.\S+|t\.me/\S+')
    text = url_pattern.sub('', text)

    # Lowercasing
    text = text.lower()

    # Removing punctuation
    text = "".join([char for char in text if char not in string.punctuation + '»«—'])

    # Removing stop words
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)

train_df_text = train_df_text.groupby('source_id').agg({
    'text': ' '.join,
    'source_category': 'first'
}).reset_index()

test_df_text = test_df_text.groupby('source_id').agg({
    'text': ' '.join
}).reset_index()

train_df_text['text_prep'] = train_df_text['text'].apply(preprocess_text)
test_df_text['text_prep'] = test_df_text['text'].apply(preprocess_text)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train = tfidf_vectorizer.fit_transform(train_df_text['text_prep'])
X_test = tfidf_vectorizer.transform(test_df_text['text_prep'])

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df_text['source_category'])

classifier = LogisticRegression(max_iter=1000)
# classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

sample_submission = test_df_text[['source_id']].copy()
sample_submission['source_category'] = label_encoder.inverse_transform(y_pred)

In [ ]:
sample_submission.set_index('source_id', inplace=True)

In [ ]:
sample_submission['source_category'].value_counts()

source_category
AGGRESSIVE_INFORMATION    984
RESTRAINED_INFORMATION    366
SAFE_CONTENT               34
UNRECOGNIZED_REPUBLICS      2
Name: count, dtype: int64

In [ ]:
sample_submission

,source_category
source_id,
6,AGGRESSIVE_INFORMATION
16,AGGRESSIVE_INFORMATION
54,AGGRESSIVE_INFORMATION
117,AGGRESSIVE_INFORMATION
128,AGGRESSIVE_INFORMATION
...,...
145007518,AGGRESSIVE_INFORMATION
145007576,AGGRESSIVE_INFORMATION
145007721,RESTRAINED_INFORMATION


In [ ]:
sample_submission.to_csv('sample_submission_gradient_encoding.csv')